In [3]:
def data_prep(data,K):
    clientID = data.ClientID.unique()
    bnlearn_data = pd.DataFrame()
    for i in range(len(clientID)):
        temp = data[data['ClientID']==clientID[i]]
        if 14 in temp.ProjectType.values:
            continue

        if len(temp)<=K:
            l = []
            for j in range(len(temp)-1,-1,-1):
                l = l+[temp['ProjectType'].iloc[j]]
            for j in range(K-len(temp)):
                l = l+[-1]
            l.insert(0,-1)
            l.insert(0,1)
            l = pd.Series(np.flip(l))
            bnlearn_data = bnlearn_data.append(l,ignore_index=True)

        elif len(temp) > K:
            a = len(temp)
            for a in range(len(temp)): 
                l = []
                if (len(temp)-K-1-a) >= -1:
                    for j in range(len(temp)-1-a,len(temp)-K-1-a,-1):
                            l = l+[temp['ProjectType'].iloc[j]]
                    if a == 0:
                        l.insert(0,-1)
                        l.insert(0,1)
                    else:
                        l.insert(0,temp['ProjectType'].iloc[len(temp)-1-a])
                        l.insert(0,0)
                elif (len(temp)-K-1-a) < -1:
                    for j in range(len(temp)-a-1,-1,-1): 
                        l = l+[temp['ProjectType'].iloc[j]]
                    for j in range((K-1)-(len(temp)-1-a)):
                        l = l+[-1]
                    l.insert(0,temp['ProjectType'].iloc[len(temp)-a])
                    l.insert(0,0)

                l = pd.Series(np.flip(l))
                bnlearn_data = bnlearn_data.append(l,ignore_index=True)


    bnlearn_data.columns = ['PT'+str(k+1) for k in range(K+1)]+['Exit']
    return bnlearn_data




In [4]:
def parameter_learning(train,K):
    print('Parameter training')
    edges = []
    for i in range(len(train.columns)):

        if train.columns[i]!='Exit':
            if train.columns[i+1]=='Exit' :
                continue
            edges = edges + [(train.columns[i],train.columns[i+1])]
        else:
            for col in train.columns.drop(['Exit',"PT"+str(K+1)]):
                edges = edges + [(col,'Exit')]
    edges = edges + [('Exit',"PT"+str(K+1))]
    DAG = bn.make_DAG(edges,methodtype='bayes',verbose = 0)
    DAG_update = bn.parameter_learning.fit(DAG, train,methodtype='maximumlikelihood',verbose=0)
    return DAG_update

In [7]:
def predict(test,DAG,K):
    print('Prediction')
    evidence_labels = test.columns.drop(["PT"+str(K+1),'Exit'])
    prediction = pd.DataFrame()
#     print(len(test))
    for i in range(len(test)):
#         print(i)
        evidence = {}
        for j in evidence_labels:
            evidence[j] = test.iloc[i][j]
        inference_exit = bn.inference.fit(DAG,variables=['Exit'],evidence=evidence,verbose=0)
        exit = inference_exit.df['Exit'].iloc[inference_exit.df.p.idxmax()]
        evidence['Exit'] = exit
        inference_pK = bn.inference.fit(DAG,variables=["PT"+str(K+1)],evidence=evidence,verbose=0)
        predict_exit = exit
        predict_pK = inference_pK.df["PT"+str(K+1)].iloc[inference_pK.df.p.idxmax()]
        prediction = prediction.append({'Predicted Exit':predict_exit,'Predicted pK':predict_pK},ignore_index=True)
    prediction['Actual exit'] = test['Exit'].reset_index(drop=True)
    prediction['Actual pK'] = test["PT"+str(K+1)].reset_index(drop=True)
    return prediction

In [4]:
def accuracy(prediction):
    Exit_accuracy = len(prediction[prediction['Predicted Exit']==prediction['Actual exit']])/len(prediction)
    pK_accuracy = len(prediction[prediction['Predicted pK']==prediction['Actual pK']])/len(prediction)
    return Exit_accuracy,pK_accuracy

In [8]:
def plot_distribution(rpath,prediction,K):
    plt.figure(figsize=(10,10))
    plt.subplot(2,2,1)
    sns.countplot(prediction['Predicted Exit'],color = 'b')
    plt.subplot(2,2,2)
    sns.countplot(prediction['Actual exit'],color = 'b')
    plt.subplot(2,2,3)
    sns.countplot(prediction['Predicted pK'],color = 'b')
    plt.subplot(2,2,4)
    sns.countplot(prediction['Actual pK'],color = 'b')
    plt.savefig(rpath+"//Distribution plot for K = "+str(K)+'.png')
    plt.close()
    

In [7]:
def uniRNDM_prediction(train,test,K):
    target_pK = np.sort(train['PT'+str(K+1)].unique())
    target_exit = np.sort(train['Exit'].unique())
    predicted_pK = np.random.choice(target_pK,len(test))
    predicted_exit = np.random.choice(target_exit,len(test))
    prediction = pd.DataFrame({'Actual exit':test['Exit'],'Predicted Exit':predicted_exit,
                               'Actual pK':test['PT'+str(K+1)],'Predicted pK':predicted_pK})
    return prediction

In [9]:
def paRNDM_prediction(train,test,K):
    target_pK = np.sort(train['PT'+str(K+1)].unique())
    prob_pK = [len(train[train['PT'+str(K+1)]==target_pK[i]])/len(train) for i in range(len(target_pK))]
    target_exit = np.sort(train['Exit'].unique())
    predicted_pK = np.random.choice(target_pK,len(test),p = prob_pK)
    predicted_exit = np.random.choice(target_exit,len(test))
    prediction = pd.DataFrame({'Actual exit':test['Exit'],'Predicted Exit':predicted_exit,
                               'Actual pK':test['PT'+str(K+1)],'Predicted pK':predicted_pK})
    return prediction

In [12]:
def LogReg_prediction(train,test,K):
    ytrain_pK = train['PT'+str(K+1)]
    ytest_pK = test['PT'+str(K+1)]
    ytrain_exit = train["Exit"]
    ytest_exit = test["Exit"]
    
    xtrain = train.drop(['PT'+str(K+1),'Exit'],axis=1).reset_index(drop=True)
    one_hot_xtrain = pd.DataFrame()
    for col in xtrain.columns:
        one_hot_xtrain = pd.concat([one_hot_xtrain,pd.get_dummies(xtrain[col],prefix = col)],axis=1)

    xtest = test.drop(['PT'+str(K+1),'Exit'],axis=1)
    one_hot_xtest = pd.DataFrame()
    for col in xtest.columns:
        one_hot_xtest = pd.concat([one_hot_xtest,pd.get_dummies(xtest[col],prefix = col)],axis=1)
    
    
    test_extra = [one_hot_xtest.columns[i] for i in range(len(one_hot_xtest.columns)) if one_hot_xtest.columns[i] not in one_hot_xtrain.columns]
    if len(test_extra)!=0:
        for t in test_extra:
            one_hot_xtrain[t] = [0 for i in range(len(one_hot_xtrain))]
            
    train_extra = [one_hot_xtrain.columns[i] for i in range(len(one_hot_xtrain.columns)) if one_hot_xtrain.columns[i] not in one_hot_xtest.columns]
    if len(train_extra)!=0:
        for t in train_extra:
            one_hot_xtest[t] = [0 for i in range(len(one_hot_xtest))]




    
    model_pK = LogisticRegression()
    model_pK.fit(one_hot_xtrain,ytrain_pK)
    predicted_pK = model_pK.predict(one_hot_xtest)

    model_exit = LogisticRegression()
    model_exit.fit(one_hot_xtrain,ytrain_exit)
    predicted_exit = model_exit.predict(one_hot_xtest)

    prediction = pd.DataFrame({'Actual exit':test['Exit'],'Predicted Exit':predicted_exit,
                               'Actual pK':test['PT'+str(K+1)],'Predicted pK':predicted_pK})
    
    return prediction
    




    

In [9]:
def RF_prediction(train,test,K):
    ytrain_pK = train['PT'+str(K+1)]
    ytest_pK = test['PT'+str(K+1)]
    ytrain_exit = train["Exit"]
    ytest_exit = test["Exit"]
    
    xtrain = train.drop(['PT'+str(K+1),'Exit'],axis=1).reset_index(drop=True)
    one_hot_xtrain = pd.DataFrame()
    for col in xtrain.columns:
        one_hot_xtrain = pd.concat([one_hot_xtrain,pd.get_dummies(xtrain[col],prefix = col)],axis=1)

    xtest = test.drop(['PT'+str(K+1),'Exit'],axis=1)
    one_hot_xtest = pd.DataFrame()
    for col in xtest.columns:
        one_hot_xtest = pd.concat([one_hot_xtest,pd.get_dummies(xtest[col],prefix = col)],axis=1)
    
    
    test_extra = [one_hot_xtest.columns[i] for i in range(len(one_hot_xtest.columns)) if one_hot_xtest.columns[i] not in one_hot_xtrain.columns]
    if len(test_extra)!=0:
        for t in test_extra:
            one_hot_xtrain[t] = [0 for i in range(len(one_hot_xtrain))]
            
    train_extra = [one_hot_xtrain.columns[i] for i in range(len(one_hot_xtrain.columns)) if one_hot_xtrain.columns[i] not in one_hot_xtest.columns]
    if len(train_extra)!=0:
        for t in train_extra:
            one_hot_xtest[t] = [0 for i in range(len(one_hot_xtest))]



    model_pK = RandomForestClassifier()
    model_pK.fit(one_hot_xtrain,ytrain_pK)
    predicted_pK = model_pK.predict(one_hot_xtest)

    model_exit = RandomForestClassifier()
    model_exit.fit(one_hot_xtrain,ytrain_exit)
    predicted_exit = model_exit.predict(one_hot_xtest)

    prediction = pd.DataFrame({'Actual exit':test['Exit'],'Predicted Exit':predicted_exit,
                               'Actual pK':test['PT'+str(K+1)],'Predicted pK':predicted_pK})
    
    return prediction



In [3]:
def recall_precision_pK(prediction,model):
    precision_recall = pd.DataFrame()
    for i in np.sort(prediction['Actual pK'].unique()):
        TP = len(prediction[(prediction['Actual pK'] == i) & (prediction['Predicted pK'] == i)])
        FP = len(prediction[(prediction['Actual pK'] != i) & (prediction['Predicted pK'] == i)])
        FN = len(prediction[(prediction['Actual pK'] == i) & (prediction['Predicted pK'] != i)])
        if TP+FP == 0:
            precision = 0
            recall = TP/(TP+FN)
        else:
            precision = TP/(TP+FP)
            recall = TP/(TP+FN)
        precision_recall = precision_recall.append({"Model":model,"K":K,"Class type":"Project type","Class":i,'Precision':precision,"Recall":recall},ignore_index=True)
    return precision_recall
    
def recall_precision_exit(prediction,model):
    precision_recall = pd.DataFrame()
    for i in np.sort(prediction['Actual exit'].unique()):
        TP = len(prediction[(prediction['Actual exit'] == i) & (prediction['Predicted Exit'] == i)])
        FP = len(prediction[(prediction['Actual exit'] != i) & (prediction['Predicted Exit'] == i)])
        FN = len(prediction[(prediction['Actual exit'] == i) & (prediction['Predicted Exit'] != i)])
        if TP+FP == 0:
            precision = 0
            recall = TP/(TP+FN)
        else:
            precision = TP/(TP+FP)
            recall = TP/(TP+FN)
        precision_recall = precision_recall.append({"Model":model,"K":K,"Class type":"Exit","Class":i,'Precision':precision,"Recall":recall},ignore_index=True)
    return precision_recall



In [ ]:
def parameter_learning_different_graph(train,K):
    print('Parameter training')
    edges = []
    for i in range(len(train.columns)):

        if train.columns[i]!='Exit':
            if train.columns[i+1]=='Exit' :
                continue
            edges = edges + [(train.columns[i],train.columns[i+1])]
        else:
            for col in train.columns.drop(['Exit',"PT"+str(K+1)]):
                edges = edges + [(col,'Exit')]
                edges = edges + [(col,"PT"+str(K+1))]
    edges = edges + [('Exit',"PT"+str(K+1))]
    DAG = bn.make_DAG(edges,methodtype='bayes',verbose = 0)
    DAG_update = bn.parameter_learning.fit(DAG, train,methodtype='maximumlikelihood',verbose=0)
    return DAG_update

In [14]:
def joint_predict(test,DAG,K):
    print('Prediction')
    evidence_labels = test.columns.drop(["PT"+str(K+1),'Exit'])
    prediction = pd.DataFrame()
#     print(len(test))
    for i in range(len(test)):
#         print(i)
        evidence = {}
        for j in evidence_labels:
            evidence[j] = test.iloc[i][j]
        inference = bn.inference.fit(DAG,variables=['Exit',"PT"+str(K+1)],evidence=evidence,verbose=0)
        exit = inference.df['Exit'].iloc[inference.df.p.idxmax()]
        predict_exit = exit
        predict_pK = inference.df["PT"+str(K+1)].iloc[inference.df.p.idxmax()]
        prediction = prediction.append({'Predicted Exit':predict_exit,'Predicted pK':predict_pK},ignore_index=True)
    prediction['Actual exit'] = test['Exit'].reset_index(drop=True)
    prediction['Actual pK'] = test["PT"+str(K+1)].reset_index(drop=True)
    return prediction